In [1]:
# Importing the Boston Dataset
from keras.datasets import boston_housing

In [3]:
# Loading the data into Training and test set

(train_data,train_targets),(test_data,test_targets) = boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


In [27]:
# imports numpy
import numpy as np

In [33]:
""" To prepare the data
We use feature-wise normalization: for each feature in the input
data (a column in the input data matrix), you subtract the mean of the 
feature and divide by the standard deviation, so that the feature is centered 
around 0 and has a unit standard deviation.
"""
# Normalizing the data

mean = np.mean(train_data,axis=0)
std = np.std(train_data,axis=0)

train_data -=mean
train_data /=std

test_data -=mean
test_data /=std

In [48]:
# Import necessary libraries

from keras import models
from keras import layers

In [53]:
# Building my model
# This function when called will train a deep neural network
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', 
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [54]:
# Using K-fold Validation because of the small amount of dataset available

K = 4  # Number of validation
num_epoch = 100    # The number of epoch
num_val_sample = len(train_data)// K   # Number of validation sample
all_scoress =[]                 # This stores the scores for each fold
for i in range(K):
    print('processing #', i)
    val_data = train_data[i * num_val_sample: (i + 1) * num_val_sample]
    val_target = train_targets[i * num_val_sample : (i + 1) * num_val_sample]
    
    partial_train_data = np.concatenate([train_data[:i * num_val_sample],
                                       train_data[(i + 1) * num_val_sample:]],
                                        axis=0)
    partial_train_target = np.concatenate([train_targets[:i * num_val_sample],
                                       train_targets[(i + 1) * num_val_sample:]],
                                         axis=0)
    
    # We call the build model function on the training data
    model = build_model()
    model.fit(partial_train_data,partial_train_target,epochs=num_epoch,
             batch_size=1, verbose=0)
    val_mse , val_mae = model.evaluate(val_data,val_target,verbose=0)
    all_scoress.append(val_mae) # appends the validation mean average error

processing # 0
processing # 1
processing # 2
processing # 3


In [58]:
# This displays the mean score of the K-fold
np.mean(all_scoress)

2.7202330231666565